In [106]:
!pip install kagglehub

In [108]:
!kaggle datasets download -d datafiniti/consumer-reviews-of-amazon-products
!unzip consumer-reviews-of-amazon-products.zip -d /content/dataset

zsh:1: command not found: kaggle
unzip:  cannot find or open consumer-reviews-of-amazon-products.zip, consumer-reviews-of-amazon-products.zip.zip or consumer-reviews-of-amazon-products.zip.ZIP.


In [110]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("datafiniti/consumer-reviews-of-amazon-products")

print("Path to dataset files:", path)

Path to dataset files: /Users/boyepanthera/.cache/kagglehub/datasets/datafiniti/consumer-reviews-of-amazon-products/versions/5


In [112]:
import pandas as pd

# Load the main file (update if needed)
file_path = 'dataset/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv'
df = pd.read_csv(file_path)

# Inspect the dataset
print("Columns in the dataset:", df.columns)
print(df.head())

Columns in the dataset: Index(['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand',
       'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer',
       'manufacturerNumber', 'reviews.date', 'reviews.dateAdded',
       'reviews.dateSeen', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs'],
      dtype='object')
                     id             dateAdded           dateUpdated  \
0  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
1  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
2  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
3  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
4  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   

                                                name       asins   brand  \
0  Amazon Kindle E-Reader 6" Wi

In [114]:
from nltk.tokenize import RegexpTokenizer

# Define a tokenizer to match words (ignores punctuations)
tokenizer = RegexpTokenizer(r'\w+')

# Preprocessing function with RegexpTokenizer
def preprocess_text(text):
    tokens = tokenizer.tokenize(str(text).lower())  # Tokenize and lowercase
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]  # Remove stopwords
    return ' '.join(tokens)

# Apply preprocessing to the 'reviews.text' column
df['processed_review'] = df['reviews.text'].apply(preprocess_text)

# Verify the new column
print(df[['reviews.text', 'processed_review']].head())

                                        reviews.text  \
0  I thought it would be as big as small paper bu...   
1  This kindle is light and easy to use especiall...   
2  Didnt know how much i'd use a kindle so went f...   
3  I am 100 happy with my purchase. I caught it o...   
4  Solid entry level Kindle. Great for kids. Gift...   

                                    processed_review  
0  thought would big small paper turn like palm t...  
1             kindle light easy use especially beach  
2  didnt know much use kindle went lower end im h...  
3  happy purchase caught sale really good price n...  
4  solid entry level kindle great kids gifted kid...  


In [116]:
import random

# Add synthetic labels (True, False, Unverifiable)
labels = ['True', 'False', 'Unverifiable']
df['label'] = [random.choice(labels) for _ in range(len(df))]

# Verify the labeled dataset
print(df[['processed_review', 'label']].head())

                                    processed_review         label
0  thought would big small paper turn like palm t...  Unverifiable
1             kindle light easy use especially beach         False
2  didnt know much use kindle went lower end im h...          True
3  happy purchase caught sale really good price n...  Unverifiable
4  solid entry level kindle great kids gifted kid...          True


In [118]:
from sklearn.model_selection import train_test_split

# Split the dataset into training and testing sets
train, test = train_test_split(df[['processed_review', 'label']], test_size=0.2, random_state=42)

# Save the splits for future use
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

print("Training and testing datasets created successfully!")

Training and testing datasets created successfully!


In [120]:
!pip install transformers

In [122]:
import pandas as pd

# Reload the dataset
file_path = 'dataset/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv'  # Update the path if needed
df = pd.read_csv(file_path)

# Verify the DataFrame
print("Columns in the dataset:", df.columns)
print(df.head())


Columns in the dataset: Index(['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand',
       'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer',
       'manufacturerNumber', 'reviews.date', 'reviews.dateAdded',
       'reviews.dateSeen', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs'],
      dtype='object')
                     id             dateAdded           dateUpdated  \
0  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
1  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
2  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
3  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
4  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   

                                                name       asins   brand  \
0  Amazon Kindle E-Reader 6" Wi

In [124]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Ensure necessary NLTK resources are available
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

# Preprocessing function
def preprocess_text(text):
    tokens = word_tokenize(str(text).lower())  # Tokenize and lowercase
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]  # Remove stopwords and non-alphabetic tokens
    return ' '.join(tokens)

# Preprocess the 'reviews.text' column
df['processed_review'] = df['reviews.text'].apply(preprocess_text)

# Verify the processed DataFrame
print(df[['reviews.text', 'processed_review']].head())

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/boyepanthera/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/boyepanthera/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                                        reviews.text  \
0  I thought it would be as big as small paper bu...   
1  This kindle is light and easy to use especiall...   
2  Didnt know how much i'd use a kindle so went f...   
3  I am 100 happy with my purchase. I caught it o...   
4  Solid entry level Kindle. Great for kids. Gift...   

                                    processed_review  
0  thought would big small paper turn like palm t...  
1             kindle light easy use especially beach  
2  didnt know much use kindle went lower end im h...  
3  happy purchase caught sale really good price n...  
4  solid entry level kindle great kids gifted kid...  


In [127]:
pip install nltk==3.8.1

Note: you may need to restart the kernel to use updated packages.


In [128]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/boyepanthera/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [137]:
import nltk

# Clear the existing NLTK data
nltk.data.path = ['/root/nltk_data']
nltk.download('punkt', force=True)  # Force a fresh download of punkt

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/boyepanthera/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [138]:
print("Columns in the DataFrame:", df.columns)

Columns in the DataFrame: Index(['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand',
       'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer',
       'manufacturerNumber', 'reviews.date', 'reviews.dateAdded',
       'reviews.dateSeen', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs'],
      dtype='object')


In [139]:
ls dataset/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv

dataset/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv


In [140]:
import pandas as pd

file_path = 'dataset/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv'  # Update this path as necessary
df = pd.read_csv(file_path)

# Display the first few rows to confirm successful loading
print(df.head())

                     id             dateAdded           dateUpdated  \
0  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
1  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
2  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
3  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   
4  AVqVGZNvQMlgsOJE6eUY  2017-03-03T16:56:05Z  2018-10-25T16:36:31Z   

                                                name       asins   brand  \
0  Amazon Kindle E-Reader 6" Wifi (8th Generation...  B00ZV9PXP2  Amazon   
1  Amazon Kindle E-Reader 6" Wifi (8th Generation...  B00ZV9PXP2  Amazon   
2  Amazon Kindle E-Reader 6" Wifi (8th Generation...  B00ZV9PXP2  Amazon   
3  Amazon Kindle E-Reader 6" Wifi (8th Generation...  B00ZV9PXP2  Amazon   
4  Amazon Kindle E-Reader 6" Wifi (8th Generation...  B00ZV9PXP2  Amazon   

                                          categories primaryCategories  \
0  Computers,Electronics Features,Tablets,

In [141]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Define preprocessing function
def preprocess_text(text):
    # Tokenize text
    tokens = word_tokenize(text)
    # Convert to lowercase
    tokens = [token.lower() for token in tokens]
    # Remove punctuation
    tokens = [token for token in tokens if token.isalnum()]
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    # Join tokens back into string
    return ' '.join(tokens)

# Apply preprocessing to 'reviews.text' column
df['processed_review'] = df['reviews.text'].apply(preprocess_text)

# Display the processed reviews
print(df[['reviews.text', 'processed_review']].head())

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/boyepanthera/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/boyepanthera/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                        reviews.text  \
0  I thought it would be as big as small paper bu...   
1  This kindle is light and easy to use especiall...   
2  Didnt know how much i'd use a kindle so went f...   
3  I am 100 happy with my purchase. I caught it o...   
4  Solid entry level Kindle. Great for kids. Gift...   

                                    processed_review  
0  thought would big small paper turn like palm t...  
1             kindle light easy use especially beach  
2  didnt know much use kindle went lower end im h...  
3  100 happy purchase caught sale really good pri...  
4  solid entry level kindle great kids gifted kid...  


In [142]:
# Define a function to assign sentiment labels
def assign_sentiment(rating):
    if rating >= 4:
        return 'positive'
    elif rating == 3:
        return 'neutral'
    else:
        return 'negative'

# Apply the function to create a new 'sentiment' column
df['sentiment'] = df['reviews.rating'].apply(assign_sentiment)

# Display the sentiment distribution
print(df['sentiment'].value_counts())

sentiment
positive    4686
neutral      197
negative     117
Name: count, dtype: int64


In [143]:
from sklearn.model_selection import train_test_split

# Split the dataset
train, test = train_test_split(df[['processed_review', 'sentiment']], test_size=0.2, random_state=42)

# Display the size of each set
print(f'Training set size: {len(train)}')
print(f'Testing set size: {len(test)}')

Training set size: 4000
Testing set size: 1000


In [144]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the training data
X_train = vectorizer.fit_transform(train['processed_review'])

# Transform the testing data
X_test = vectorizer.transform(test['processed_review'])

# Display the shape of the transformed data
print(f'X_train shape: {X_train.shape}')
print(f'X_test shape: {X_test.shape}')

X_train shape: (4000, 4721)
X_test shape: (1000, 4721)


In [145]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Initialize the model
model = LogisticRegression(max_iter=1000)

# Train the model
model.fit(X_train, train['sentiment'])

# Predict on the test set
predictions = model.predict(X_test)

# Evaluate the model
print(classification_report(test['sentiment'], predictions))

              precision    recall  f1-score   support

    negative       0.00      0.00      0.00        24
     neutral       1.00      0.03      0.05        39
    positive       0.94      1.00      0.97       937

    accuracy                           0.94      1000
   macro avg       0.65      0.34      0.34      1000
weighted avg       0.92      0.94      0.91      1000



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [146]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')

# Load the dataset
file_path = 'dataset/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv'
df = pd.read_csv(file_path)

# Define preprocessing function
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    # Tokenize
    words = word_tokenize(text)
    # Remove punctuation and stopwords
    words = [word for word in words if word.isalpha() and word not in stopwords.words('english')]
    # Join words back into a single string
    return ' '.join(words)

# Apply preprocessing
df['processed_review'] = df['reviews.text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/boyepanthera/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/boyepanthera/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [147]:
if 'processed_review' in df.columns:
    print(df['processed_review'].head())
else:
    print("Column 'processed_review' does not exist.")

0    thought would big small paper turn like palm t...
1               kindle light easy use especially beach
2    didnt know much use kindle went lower end im h...
3    happy purchase caught sale really good price n...
4    solid entry level kindle great kids gifted kid...
Name: processed_review, dtype: object


In [148]:
import pandas as pd

# Define a function to map ratings to sentiments
def map_rating_to_sentiment(rating):
    if rating in [4, 5]:
        return 'Positive'
    elif rating == 3:
        return 'Neutral'
    elif rating in [1, 2]:
        return 'Negative'
    else:
        return 'Unknown'  # Handle unexpected ratings

# Apply the function to create the 'label' column
df['label'] = df['reviews.rating'].apply(map_rating_to_sentiment)

In [149]:
from sklearn.model_selection import train_test_split

# Assuming 'df' is your DataFrame and 'processed_review' contains the preprocessed text
X_train, X_test, y_train, y_test = train_test_split(
    df['processed_review'], df['label'], test_size=0.2, random_state=42
)

In [150]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the training data; transform the test data
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [151]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Initialize classifiers
classifiers = {
    'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Support Vector Machine': SVC(),
    'Random Forest': RandomForestClassifier()
}

# Train and evaluate each classifier
for name, clf in classifiers.items():
    clf.fit(X_train_tfidf, y_train)
    y_pred = clf.predict(X_test_tfidf)
    print(f'--- {name} ---')
    print(classification_report(y_test, y_pred))

--- Naive Bayes ---
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00        24
     Neutral       0.00      0.00      0.00        39
    Positive       0.94      1.00      0.97       937

    accuracy                           0.94      1000
   macro avg       0.31      0.33      0.32      1000
weighted avg       0.88      0.94      0.91      1000

--- Logistic Regression ---
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00        24
     Neutral       1.00      0.03      0.05        39
    Positive       0.94      1.00      0.97       937

    accuracy                           0.94      1000
   macro avg       0.65      0.34      0.34      1000
weighted avg       0.92      0.94      0.91      1000



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-pa

--- Support Vector Machine ---
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00        24
     Neutral       1.00      0.05      0.10        39
    Positive       0.94      1.00      0.97       937

    accuracy                           0.94      1000
   macro avg       0.65      0.35      0.36      1000
weighted avg       0.92      0.94      0.91      1000



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


--- Random Forest ---
              precision    recall  f1-score   support

    Negative       1.00      0.29      0.45        24
     Neutral       1.00      0.26      0.41        39
    Positive       0.95      1.00      0.98       937

    accuracy                           0.95      1000
   macro avg       0.98      0.52      0.61      1000
weighted avg       0.96      0.95      0.94      1000



In [169]:
print(df.columns)

Index(['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand',
       'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer',
       'manufacturerNumber', 'reviews.date', 'reviews.dateAdded',
       'reviews.dateSeen', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs',
       'processed_review', 'label'],
      dtype='object')


In [173]:
import pickle
import os
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Create model directory if it doesn't exist
os.makedirs('model', exist_ok=True)

# First, save the current state of your model and vectorizer
print("Saving model and vectorizer...")

# Save the trained model (assuming your best model was Random Forest)
best_model = model  # This should be your trained Random Forest model
with open('model/sentiment_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)

# Save the vectorizer with its vocabulary
with open('model/vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

# Save feature names for verification
feature_names = vectorizer.get_feature_names_out()
with open('model/feature_names.pkl', 'wb') as f:
    pickle.dump(feature_names, f)

print("Model and vectorizer saved. Now testing...")

# Test loading and prediction
def preprocess_text(text):
    """Match the preprocessing used in training"""
    # Convert to lowercase
    text = text.lower()
    # Tokenize
    words = word_tokenize(text)
    # Remove non-alphabetic tokens and stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.isalpha() and word not in stop_words]
    return ' '.join(words)

# Load the saved components
with open('model/sentiment_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

with open('model/vectorizer.pkl', 'rb') as f:
    loaded_vectorizer = pickle.load(f)

with open('model/feature_names.pkl', 'rb') as f:
    saved_features = pickle.load(f)

# Verify feature count matches
print(f"Number of features in vectorizer: {len(loaded_vectorizer.get_feature_names_out())}")
print(f"Number of features model expects: {loaded_model.n_features_in_}")

# Test with a sample review
sample_reviews = [
    "This product is amazing and works perfectly!",
    "Not worth the money, very disappointed",
    "Good quality but a bit expensive"
]

print("\nTesting predictions...")
for review in sample_reviews:
    # Preprocess the review
    processed_review = preprocess_text(review)
    
    # Transform using loaded vectorizer
    vectorized = loaded_vectorizer.transform([processed_review])
    
    # Verify feature count
    print(f"\nOriginal review: {review}")
    print(f"Processed review: {processed_review}")
    print(f"Vectorized shape: {vectorized.shape}")
    
    # Make prediction
    try:
        prediction = loaded_model.predict(vectorized)
        probability = loaded_model.predict_proba(vectorized)
        print(f"Prediction: {prediction[0]}")
        print(f"Confidence: {np.max(probability[0]):.2f}")
    except Exception as e:
        print(f"Prediction failed: {str(e)}")

print("\nSave and test complete!")

Saving model and vectorizer...
Model and vectorizer saved. Now testing...
Number of features in vectorizer: 4596
Number of features model expects: 4721

Testing predictions...

Original review: This product is amazing and works perfectly!
Processed review: product amazing works perfectly
Vectorized shape: (1, 4596)
Prediction failed: X has 4596 features, but LogisticRegression is expecting 4721 features as input.

Original review: Not worth the money, very disappointed
Processed review: worth money disappointed
Vectorized shape: (1, 4596)
Prediction failed: X has 4596 features, but LogisticRegression is expecting 4721 features as input.

Original review: Good quality but a bit expensive
Processed review: good quality bit expensive
Vectorized shape: (1, 4596)
Prediction failed: X has 4596 features, but LogisticRegression is expecting 4721 features as input.

Save and test complete!
